In [2]:
import create_dataset
from torchvision.transforms import ToTensor
import importlib
importlib.reload(create_dataset)

data_small=create_dataset.DatasetFromSeparatePatches("addoration_of_lamb/images/",number_of_patch_side=3,transform=ToTensor())

save_folder="test"

In [3]:

from torch.utils.data import DataLoader
dataloader=DataLoader(data_small, batch_size=1,drop_last=False)



In [4]:
from reco_adapted.lightning_model import ema_segmentation_model
ckpt_path="lightning_logs/version_17/checkpoints/epoch=5-step=1199.ckpt"

model = ema_segmentation_model.load_from_checkpoint(ckpt_path,hparams_file="lightning_logs/version_25/hparams.yaml")


model parameters : 
"apply_aug":        cutmix
"apply_reco":       False
"lr":               0.0025
"max_epochs":       20
"num_negatives":    512
"num_queries":      256
"num_segments":     2
"optimizer":        AdamW
"output_dim":       256
"strong_threshold": 0.0
"temp":             0.5
"weak_threshold":   0.7
"weight_decay":     0.0005


In [7]:
import torch
import pytorch_lightning as pl
from PIL import Image
import numpy as np
from tqdm import tqdm

#batch=next(iter(dataloader))
seg_model=model.model
path_save="test/"   
seg_model.eval()
seg_model.to("cuda")
with torch.no_grad():
        
    
    i=0
    print(len(dataloader))
    for batch in tqdm(dataloader):
       
        batch=batch.to("cuda")
        data_small.save(i,path_save)
        prediction=seg_model(batch)
        pred_labels=torch.softmax(prediction.cpu(), dim=1)[:,1]
        
        Image.fromarray((pred_labels.squeeze(0).unsqueeze(-1).expand(-1,-1,3).numpy()*255).astype(np.uint8)).save(path_save+f"{i}.jpg")
        i=i+1

9144


  9%|▊         | 782/9144 [05:01<53:44,  2.59it/s]  


KeyboardInterrupt: 